In [ ]:
import os
import sys
import rioxarray as rxr
import numpy as np
import treelite
import rasterio as rio
import matplotlib
import geopandas as gpd
import pandas as pd

from osgeo import gdal,gdal_array,osr

import matplotlib.pyplot as plt

import panel as pn

from tqdm import tqdm

import holoviews as hv
import geoviews as gv
gv.extension('bokeh')
#gv.extension('matplotlib')

import xarray as xr
import rioxarray as rxr

import joblib

import glob

#save the plots for comparison
import cartopy.crs as ccrs
from bokeh.resources import INLINE

#sys.path.append('/home/jovyan/BOREAL_MD/rfexpl')
#import rfexpl

import warnings
warnings.filterwarnings("ignore")

#shap.initjs()

In [ ]:
gv.__version__

In [ ]:
## Define global variables

#Circumpolar
#minlon = -179.02
#maxlon = 180
#minlat = 49.99
#maxlat = 74.95

#Eurasia
#minlon = 25
#maxlon = 179.02
#minlat = 52.01
#maxlat = 75.00


#04/17/2023 Boreal
minlon = -169
maxlon = -50
minlat = 45
maxlat = 75

bbox = [minlon, minlat, maxlon, maxlat]
#fn = '/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/wc2.1_2.5m_bio_1.tif'
fn = '/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim_cmip6/wc2.1_2.5m_bio/wc2.1_2.5m_bio_1.tif'
tmp = rxr.open_rasterio(fn, mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat)
lons = tmp.x.values
lats = tmp.y.values
shape = (len(lats), len(lons))
print('Expected size:')
print(shape)
tmp=None

print('Lons:')
print(len(lons))
with rio.open(fn) as src:
    meta = src.meta
# with rxr.open_rasterio(fn) as src:
#     meta = src.meta
#print(meta)
src.close()
# color map for prediction
## 'black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850'
clrs = ['#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850']
cmap_p = matplotlib.colors.LinearSegmentedColormap.from_list(name="color_p", 
                                                             colors=clrs, N=16) 

# color map for trend
brbg= matplotlib.cm.get_cmap('BrBG', 256)
grey = matplotlib.cm.get_cmap('Greys', 20)
gs = grey(np.linspace(0, 1, 20))
newcolors = brbg(np.linspace(0, 1, 256))
newcolors[85:170, :] = gs[5]
cmap_t = matplotlib.colors.ListedColormap(newcolors)

# set basemap
#baseMap = gv.tile_sources.EsriImagery
baseMap = gv.tile_sources.CartoLight
#baseMapRGB = gv.util.get_tile_rgb(baseMap,bbox=(minlon, minlat, maxlon, maxlat), zoom_level=1).opts(width=800, height=400, projection=ccrs.PlateCarree())


In [ ]:
#boreal extent
boreal_ext = gpd.read_file('/panfs/ccds02/nobackup/people/pmontesa/userfs02/arc/wwf_circumboreal_Dissolve.shp')
NA_boreal_ext = boreal_ext[boreal_ext["REALM"] == "NA"]
# fig, ax = plt.subplots(figsize = (10,10))
# NA_boreal_ext.plot(ax=ax)
#plt.show()

NA_boreal_ext = NA_boreal_ext.to_crs(4326) #Set CRS

In [ ]:
#Add 30 arc sec (about 100 km) buffer
NA_boreal_ext_buff = NA_boreal_ext.buffer(0.833)
fig, ax = plt.subplots(figsize = (10,10))
NA_boreal_ext.plot(ax=ax)
NA_boreal_ext_buff.plot(ax=ax, color = 'green', alpha = .5)
plt.show()

In [ ]:
#boreal regions
wwf_reg = gpd.read_file('/panfs/ccds02/nobackup/people/pmontesa/userfs02/arc/wwf_terr_ecos.shp')
boreal_reg = gpd.clip(wwf_reg, NA_boreal_ext)
boreal_reg = boreal_reg.dissolve(by='ECO_NAME')
fig, ax = plt.subplots(figsize = (10,10))
boreal_reg.plot(ax=ax, cmap='tab20')
plt.title("WWF Ecoregions")
plt.show()
print("The number of regions is: "+str(len(boreal_reg)))
#print(boreal_reg.head())

In [ ]:
#Save ecoregions into list
import pandas as pd
df = pd.DataFrame(boreal_reg)
eco_reg_list = list(df.index.values)

In [ ]:
# funtion to plot result 
def map_bioclim(arr, mode="pred"):
    if mode == "pred":
        cmap = cmap_p
        clim = (0.5, 25.5)
        logz = True
    if mode == "diff":
        cmap = "BrBG"
        clim = (-5.0, 5.0)
        logz = False
    if mode == "trend":
        cmap = cmap_t
        clim = (-5.0, 5.0)
        logz = False
        
    img_opts = dict(
        width=600, 
        height=300, 
        logz=logz,
        cmap=cmap,
        colorbar=True,
        clim = clim,
        tools=["hover"], active_tools=['wheel_zoom']
        )
    gv_dataset = gv_dataset = gv.Dataset((lons, lats, arr), ['longitude', 'latitude'], mode)
    
    return gv.Image(gv_dataset).opts(**img_opts)

In [ ]:
fp = '/explore/nobackup/people/mfrost2/projects/boreal_hcan/output/'
os.chdir(fp)

#sn_list = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
sn_list = ['ssp370', 'ssp585']
#sn = sn_list[1]

In [ ]:
os.getcwd()

In [ ]:
# show current
tmp = rxr.open_rasterio('North_America_Current_filtered_noPerm_20230425.tif', mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()

img0 = map_bioclim(tmp, 'pred')
pn.Column(img0*baseMap)

In [ ]:
# show current clipped
tmp = rxr.open_rasterio('North_America_Current_filtered_noPerm_20230425.tif', mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
tmp_clipped = tmp.rio.clip(NA_boreal_ext.geometry, drop = False)

img0 = map_bioclim(tmp_clipped, 'pred')
pn.Column(img0*baseMap)

In [ ]:
# show current clipped by region
#tmp = rxr.open_rasterio('North_America_Current_filtered_noPerm_20230425.tif', mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
tmp_r_clipped = tmp.rio.clip(boreal_reg.geometry, drop = False)

img0 = map_bioclim(tmp_r_clipped, 'pred')
pn.Column(img0*baseMap)

# Load Permafrost zone types
### (Extent - Ice Content - Landform)

In [ ]:
#Permafrost combination types,
pf_reg = gpd.read_file(r'/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/GIS/permafrost_combo.shp') 
#pf_reg = gpd.read_file(r'/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/GIS/Permafrost_extent.shp')
pf_reg = gpd.clip(pf_reg, NA_boreal_ext)
#boreal_reg = boreal_reg.dissolve(by='ECO_NAME') ALready performed in ARCGIS
fig, ax = plt.subplots(figsize = (10,10))
pf_reg.plot(ax=ax, cmap='tab20')
plt.title("Permafrost Zones")
plt.show()
print("The number of zones is: "+str(len(pf_reg)))
#print(boreal_reg.head())

In [ ]:
tmp_pf_clipped = tmp.rio.clip(pf_reg.geometry, drop = False)

img0 = map_bioclim(tmp_pf_clipped, 'pred')
pn.Column(img0*baseMap)

## Load Files to graph

In [ ]:
# show maps
med_list = []
med_list_diff = []
for j in range(0, len(sn_list)):
    sn = sn_list[j]
    
    #Make lists of medians and median differences

    for file in glob.glob('Median_'+sn+'*0.tif'):
        med_list.append(file)
    med_list = sorted(med_list)

    for file in glob.glob('Median_'+sn+'*F.tif'):
        med_list_diff.append(file)
    med_list_diff = sorted(med_list_diff)
      
    for i in range(0, len(med_list)):
        
        yr = med_list[i][14:23]

        img_p = rxr.open_rasterio(med_list[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        pimg = map_bioclim(img_p, 'pred')
        title = f"Predicted HCan {yr} {sn}"
        #title = f"Predicted HCan {sn[i]}"
        pimg.opts(title=title)

        img_d = rxr.open_rasterio(med_list_diff[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        dimg = map_bioclim(img_d, 'diff')
        title = f"HCan difference CMIP6 {sn} vs. Current"
        dimg.opts(title=title)
    
#    rows.append(pimg*baseMap+dimg*baseMap)
    
print(med_list)
print(med_list_diff)

In [ ]:
# show maps
rows = []

for i in range(0, len(med_list)):

    yr = med_list[i][14:23]
    sn1 = med_list[i][7:13]
    
    img_p = rxr.open_rasterio(med_list[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
    #clip by NA_boreal_ext
    img_p = img_p.rio.clip(NA_boreal_ext_buff.geometry, drop = False)
    pimg = map_bioclim(img_p, 'pred')
    title = f"Predicted HCan {yr} {sn1}"
    #title = f"Predicted HCan {sn[i]}"
    pimg.opts(title=title)

    img_d = rxr.open_rasterio(med_list_diff[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
    #clip by NA_boreal_ext
    img_d = img_d.rio.clip(NA_boreal_ext_buff.geometry, drop = False)
    dimg = map_bioclim(img_d, 'diff')
    title = f"HCan difference CMIP6 {sn1} vs. Current"
    dimg.opts(title=title)
    
    rows.append(pimg*baseMap+dimg*baseMap)

In [ ]:
rows

# Histograms of scenarios

In [ ]:
for i in range(0, len(med_list)):
    fig, axes = plt.subplots(1, 2, figsize = (10,5), sharey = True)
    
    #h_can charts
    yr = med_list[i][14:23]
    sn1 = med_list[i][7:13]
    
    tmp_p = rxr.open_rasterio(med_list[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
    valMax = str(round(np.nanmax(tmp_p),3))
    valMin = str(round(np.nanmin(tmp_p),3))
    valMean = str(round(np.nanmean(tmp_p),3))
    valSTD = str(round(np.nanstd(tmp_p),3))

    plt1 = tmp_p.plot.hist(ax = axes[0], bins=60, range=[-.5,30.5], color = "forestgreen", edgecolor = 'black')
    axes[0].set(title=f'Predicted HCan {yr} {sn1}',
    xlabel='Canopy Height (meters)',
    ylabel='Frequency')
    plt.text(0.5,0.5, f'mean: {valMean}\n STD: {valSTD}\n Min: {valMin}\n Max: {valMax}', fontsize=14, transform = axes[0].transAxes)
    
    #difference charts
    yr = med_list_diff[i][14:23]
    
    tmp_d = rxr.open_rasterio(med_list_diff[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
    valMax = str(round(np.nanmax(tmp_d),3))
    valMin = str(round(np.nanmin(tmp_d),3))
    valMean = str(round(np.nanmean(tmp_d),3))
    valSTD = str(round(np.nanstd(tmp_d),3))

    plt2 = tmp_d.plot.hist(ax = axes[1], bins=60, range=[-30.5,30.5], color = "orange", edgecolor = 'black')
    axes[1].set(title=f'HCan diff prediction {yr} {sn1} vs Current',
    xlabel='Canopy height (meters)',
    ylabel='Frequency')
    plt.text(0.6,0.5, f'mean: {valMean}\n STD: {valSTD}\n Min: {valMin}\n Max: {valMax}', fontsize=14, transform = axes[1].transAxes)
    
    plt.show()
#    rows.append(tmp_p)
#    rows.append(plt1 + plt2)

# Summarize by region

In [ ]:
# ##Test to see if regions functions worked
# boreal_reg_temp = boreal_reg[boreal_reg.index == eco_reg_list[14]]
# fig, ax = plt.subplots(figsize = (10,10))
# boreal_reg_temp.plot(ax=ax, cmap='tab20')
# plt.show()


In [ ]:
boreal_reg.index

In [ ]:
#Make histograms based on WWF ECORegions

#eco_reg_small = eco_reg_list[0:3]
#for j in range(0, len(eco_reg_small)):
for j in range(0, len(eco_reg_list)):
    boreal_reg_temp = boreal_reg[boreal_reg.index == eco_reg_list[j]]
    for i in range(0, len(med_list)):
        fig, axes = plt.subplots(1, 2, figsize = (10,5), sharey = True)

        #h_can charts
        yr = med_list[i][14:23]
        sn1 = med_list[i][7:13]
        reg = eco_reg_list[j]

        tmp_p = rxr.open_rasterio(med_list[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        tmp_p = tmp_p.rio.clip(boreal_reg_temp.geometry, drop = False)
        
        valMax = str(round(np.nanmax(tmp_p),3))
        valMin = str(round(np.nanmin(tmp_p),3))
        valMean = str(round(np.nanmean(tmp_p),3))
        valSTD = str(round(np.nanstd(tmp_p),3))
        valSize = str(np.count_nonzero(~np.isnan(tmp_p)))

        plt1 = tmp_p.plot.hist(ax = axes[0], bins=60, range=[-.5,30.5], color = "forestgreen", edgecolor = 'black', label = 'test')
       
        axes[0].set(title=f'Predicted HCan {yr} {sn1}',
           #         subtitle='test',
           xlabel='Canopy Height (meters)',
           ylabel='Frequency')
        plt.text(0.5,0.5, f'Region: {reg}\n mean: {valMean}\n STD: {valSTD}\n Min: {valMin}\n Max: {valMax}\n Cells: {valSize}', fontsize=14, transform = axes[0].transAxes)
        
        #difference charts
        yr = med_list_diff[i][14:23]

        tmp_d = rxr.open_rasterio(med_list_diff[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        tmp_d = tmp_d.rio.clip(boreal_reg_temp.geometry, drop = False)
        
        valMaxD = str(round(np.nanmax(tmp_d),3))
        valMinD = str(round(np.nanmin(tmp_d),3))
        valMeanD = str(round(np.nanmean(tmp_d),3))
        valSTDD = str(round(np.nanstd(tmp_d),3))
        valSizeD = str(np.count_nonzero(~np.isnan(tmp_d)))

        plt2 = tmp_d.plot.hist(ax = axes[1], bins=60, range=[-30.5,30.5], color = "orange", edgecolor = 'black')
        axes[1].set(title=f'HCan diff prediction {yr} {sn1} vs Current',
           xlabel='Canopy height (meters)',
           ylabel='Frequency')
        plt.text(0.6,0.5, f'Region: {reg}\n mean: {valMeanD}\n STD: {valSTDD}\n Min: {valMinD}\n Max: {valMaxD}\n Cells: {valSizeD}', fontsize=14, transform = axes[1].transAxes)

#        plt.show()
        
         
        print(np.count_nonzero(~np.isnan(tmp_p)))

In [ ]:
boreal_pred_df = pd.DataFrame()
boreal_diff_df = pd.DataFrame()
##Make DataFrame
#for j in range(0, len(eco_reg_small)):
for j in range(0, len(eco_reg_list)):
    boreal_reg_temp = boreal_reg[boreal_reg.index == eco_reg_list[j]]
    for i in range(0, len(med_list)):

        #h_can 
        yr = med_list[i][14:23]
        sn1 = med_list[i][7:13]
        reg = eco_reg_list[j]

        tmp_p = rxr.open_rasterio(med_list[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        tmp_p = tmp_p.rio.clip(boreal_reg_temp.geometry, drop = False)

      
        #difference charts
        tmp_d = rxr.open_rasterio(med_list_diff[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        tmp_d = tmp_d.rio.clip(boreal_reg_temp.geometry, drop = False)
        
        #make df
        pred_df = pd.DataFrame(tmp_p)
        pred_df = pred_df.stack().reset_index()
        pred_df['year'] = yr
        pred_df['ssp'] = sn1
        pred_df['reg'] = reg
        boreal_pred_df = pd.concat([boreal_pred_df,pred_df])
        
        diff_df = pd.DataFrame(tmp_d)
        diff_df = diff_df.stack().reset_index()
        diff_df['year'] = yr
        diff_df['ssp'] = sn1
        diff_df['reg'] = reg
        boreal_diff_df = pd.concat([boreal_diff_df,diff_df])


In [ ]:
#Save as CSV
boreal_pred_df = boreal_pred_df.rename(columns={
    "level_0": "vert", 
    "level_1": "horiz",
    0: "hcan"})
boreal_pred_df = boreal_pred_df.sort_values('year')
boreal_pred_df.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_region_pred.csv',index=False)

boreal_diff_df = boreal_diff_df.rename(columns={
    "level_0": "vert", 
    "level_1": "horiz",
    0: "hcan_diff"})
boreal_diff_df = boreal_diff_df.sort_values('year')

boreal_diff_df.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_region_diff.csv',index=False)

In [ ]:
#count of pixels
pixel_counts = boreal_pred_df.groupby(['reg']).hcan.agg(['count']).sort_values('count').reset_index()
pixel_counts['count'] = (pixel_counts['count'] / 8).astype(int)
print(pixel_counts)

In [ ]:
#Boreal Prediciton aggregate
boreal_pred_agg = boreal_pred_df.groupby(['ssp','year','reg']).hcan.agg(['min', 'max','mean','median','std','count']).sort_values('year').reset_index()
boreal_pred_agg.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_region_pred_agg.csv',index=False)
boreal_pred_agg = boreal_pred_agg.sort_values('year')
boreal_pred_agg.head()

In [ ]:
#Boreal Difference aggregate
boreal_diff_agg = boreal_diff_df.groupby(['ssp','year','reg']).hcan_diff.agg(['min', 'max','mean','median','std','count']).sort_values('year').reset_index()
boreal_diff_agg.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_region_diff_agg.csv',index=False)
boreal_diff_agg = boreal_diff_agg.sort_values('year')
boreal_diff_agg.head()

In [ ]:
#Prediction graphs

import seaborn as sns

g = sns.FacetGrid(boreal_pred_agg, col="ssp", height=8)
g.map_dataframe(sns.barplot, x = 'reg', y = 'mean', hue='year', palette= 'tab10')
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle('Boreal Canopy Height Prediction by Region {ssp}')
g.add_legend()
g.set_xticklabels(rotation=90)
g.set_xlabels("Region")
g.set_ylabels("Mean Conopy Height (m)")
#add overall title

plt.show()

In [ ]:
#Difference Graphs
g = sns.FacetGrid(boreal_diff_agg, col="ssp", height=8)
g.map_dataframe(sns.barplot, x = 'reg', y = 'mean', hue='year', palette= 'tab10')
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle('Boreal Canopy Height Differences Prediction to Current by Permafrost Zone {ssp}')
g.add_legend()
g.set_xticklabels(rotation=90)
g.set_xlabels("Region")
g.set_ylabels("Mean Conopy Height differences (m)")
#add overall title

plt.show()

# Summarize by Permafrost Zone

In [ ]:
#create subcategories based on extent, ice content, and landform
pf_reg['extent'] = pf_reg['COMBO'].str[0]
pf_reg['ice_content'] = pf_reg['COMBO'].str[1]
pf_reg['landform'] = pf_reg['COMBO'].str[2]
pf_reg.head()

In [ ]:
##could also look at extent, ice_content, or landform instead of combo
pf_list = pf_reg.COMBO.unique().tolist()
pf_list

In [ ]:
# ##Test to see if pf functions worked
# pf_temp = pf_reg_temp = pf_reg[pf_reg.COMBO == pf_list[1]]
# fig, ax = plt.subplots(figsize = (10,10))
# pf_temp.plot(ax=ax, cmap='tab20')
# plt.show()


In [ ]:
#create dataframes for each Permafrost Zone
pf_pred_df = pd.DataFrame()
pf_diff_df = pd.DataFrame()
##Make DataFrame
#for j in range(0, len(eco_reg_small)):
for j in range(0, len(pf_list)):
    pf_reg_temp = pf_reg[pf_reg.COMBO == pf_list[j]]
    for i in range(0, len(med_list)):

        #h_can 
        yr = med_list[i][14:23]
        sn1 = med_list[i][7:13]
        pf = pf_list[j]

        tmp_p = rxr.open_rasterio(med_list[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        tmp_p = tmp_p.rio.clip(pf_reg_temp.geometry, drop = False)

      
        #difference charts
        tmp_d = rxr.open_rasterio(med_list_diff[i], mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).squeeze()
        tmp_d = tmp_d.rio.clip(pf_reg_temp.geometry, drop = False)
        
        #make df
        pred_df = pd.DataFrame(tmp_p)
        pred_df = pred_df.stack().reset_index()
        pred_df['year'] = yr
        pred_df['ssp'] = sn1
        pred_df['pf_codes'] = pf
        pf_pred_df = pd.concat([pf_pred_df,pred_df])
        
        diff_df = pd.DataFrame(tmp_d)
        diff_df = diff_df.stack().reset_index()
        diff_df['year'] = yr
        diff_df['ssp'] = sn1
        diff_df['pf_codes'] = pf
        pf_diff_df = pd.concat([pf_diff_df,diff_df])

In [ ]:
pf_pred_df.head()

In [ ]:
#Save as CSV
pf_pred_df = pf_pred_df.rename(columns={
    "level_0": "vert", 
    "level_1": "horiz",
    0: "hcan"})
pf_pred_df['extent'] = pf_pred_df['pf_codes'].str[0]
pf_pred_df['ice_content'] = pf_pred_df['pf_codes'].str[1]
pf_pred_df['landform'] = pf_pred_df['pf_codes'].str[2]
pf_pred_df = pf_pred_df.sort_values('year')

pf_pred_df.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_pf_pred.csv',index=False)

pf_diff_df = pf_diff_df.rename(columns={
    "level_0": "vert", 
    "level_1": "horiz",
    0: "hcan_diff"})
pf_diff_df['extent'] = pf_diff_df['pf_codes'].str[0]
pf_diff_df['ice_content'] = pf_diff_df['pf_codes'].str[1]
pf_diff_df['landform'] = pf_diff_df['pf_codes'].str[2]
pf_diff_df = pf_diff_df.sort_values('year')

boreal_diff_df.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_pf_diff.csv',index=False)

In [ ]:
#count of pixels
pixel_counts = pf_pred_df.groupby(['pf_codes']).hcan.agg(['count']).sort_values('count').reset_index()
pixel_counts['count'] = (pixel_counts['count'] / 8).astype(int)
print(pixel_counts)

In [ ]:
#Boreal Prediciton aggregate
pf_pred_agg = pf_pred_df.groupby(['ssp','year','pf_codes', 'extent', 'ice_content', 'landform']).hcan.agg(['min', 'max','mean','median','std','count']).sort_values('year').reset_index()
pf_pred_agg.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_pf_pred_agg.csv',index=False)
pf_pred_agg = pf_pred_agg.sort_values('year')
pf_pred_agg.head()

In [ ]:
#Boreal Difference aggregate
pf_diff_agg = pf_diff_df.groupby(['ssp','year','pf_codes', 'extent', 'ice_content', 'landform']).hcan_diff.agg(['min', 'max','mean','median','std','count']).sort_values('year').reset_index()
pf_diff_agg.to_csv('/panfs/ccds02/nobackup/people/mfrost2/projects/boreal_hcan/data/boreal_pf_diff_agg.csv',index=False)
pf_diff_agg = pf_diff_agg.sort_values('year')
pf_diff_agg.head()

In [ ]:
#Prediction graphs

import seaborn as sns

g = sns.FacetGrid(pf_pred_agg, col="ssp", height=8)
g.map_dataframe(sns.barplot, x = 'pf_codes', y = 'mean', hue='year', palette= 'tab10')
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle('Boreal Canopy Height Prediction by Permafrost Zone {ssp}')
g.add_legend()
g.set_xticklabels(rotation=90)
g.set_xlabels("Region")
g.set_ylabels("Mean Conopy Height (m)")
#add overall title

plt.show()

In [ ]:
#Difference Graphs
g = sns.FacetGrid(pf_diff_agg, col="ssp", height=8)
g.map_dataframe(sns.barplot, x = 'pf_codes', y = 'mean', hue='year', palette= 'tab10')
g.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
g.fig.suptitle('Boreal Canopy Height Differences Prediction to Current by Permafrost Zone {ssp}')
g.add_legend()
g.set_xticklabels(rotation=90)
g.set_xlabels("Region")
g.set_ylabels("Mean Conopy Height differences (m)")
#add overall title

plt.show()

In [ ]:
yr_list = ['2021-2040','2041-2060','2061-2080', '2081-2100']